In [46]:
import numpy as np
import jax.numpy as jnp
import jax
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

%matplotlib inline

is_dark = True
theme, cs = rp.mpl_setup(is_dark)
rp.plotly_setup(is_dark)

In [47]:
%cd ~/cdv

/home/nmiklaucic/cdv


In [48]:
from jarvis.db.figshare import data

data3d = data('dft_3d')

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.


In [49]:
df = pd.DataFrame(data3d)
df.head()

,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,...,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0...",na,0.0,...,5.956,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti
1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]...",na,0.0,...,5.522,na,"[OPT-LOPTICS,JVASP-86097.zip,https://ndownload...",7,na,na,na,na,mp-568319,-B-Dy
2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"{'lattice_mat': [[-1.833590720595598, 1.833590...",na,0.0,...,10.960,na,"[OPT-LOPTICS,JVASP-64906.zip,https://ndownload...",4,na,na,na,na,auid-3eaf68dd483bf4f4,-Be-Os-Ru
3,JVASP-98225,14,P2_1/c,KBi,-0.44140,OptB88vdW,0.472,"{'lattice_mat': [[7.2963518353359165, 0.0, 0.0...",na,0.0,...,5.145,na,[],32,na,na,na,na,mp-31104,-Bi-K
4,JVASP-10,164,P-3m1,VSe2,-0.71026,OptB88vdW,0.000,"{'lattice_mat': [[1.6777483798834445, -2.90594...",na,0.0,...,5.718,0.23,"[FD-ELAST,JVASP-10.zip,https://ndownloader.fig...",3,48.79,33.05,0.0,na,mp-694,-Se-V


In [50]:
from jarvis.core.atoms import Atoms
from pymatgen.io.jarvis import JarvisAtomsAdaptor

df['atoms'] = [JarvisAtomsAdaptor.get_structure(Atoms.from_dict(a)) for a in df['atoms']]
df.head()

,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,...,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",na,0.0,...,5.956,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti
1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",na,0.0,...,5.522,na,"[OPT-LOPTICS,JVASP-86097.zip,https://ndownload...",7,na,na,na,na,mp-568319,-B-Dy
2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",na,0.0,...,10.960,na,"[OPT-LOPTICS,JVASP-64906.zip,https://ndownload...",4,na,na,na,na,auid-3eaf68dd483bf4f4,-Be-Os-Ru
3,JVASP-98225,14,P2_1/c,KBi,-0.44140,OptB88vdW,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",na,0.0,...,5.145,na,[],32,na,na,na,na,mp-31104,-Bi-K
4,JVASP-10,164,P-3m1,VSe2,-0.71026,OptB88vdW,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",na,0.0,...,5.718,0.23,"[FD-ELAST,JVASP-10.zip,https://ndownloader.fig...",3,48.79,33.05,0.0,na,mp-694,-Se-V


In [51]:
for col in df.columns:
    if 'na' in df[col].values:
        try:
            df[col] = pd.to_numeric(df[col].replace('na', np.nan))
        except (TypeError, ValueError):
            print(col)

df.select_dtypes(include='number')

/tmp/ipykernel_3393550/1356337930.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = pd.to_numeric(df[col].replace('na', np.nan))


elastic_tensor
modes
dimensionality


,formation_energy_peratom,optb88vdw_bandgap,slme,magmom_oszicar,spillage,kpoint_length_unit,maxdiff_mesh,maxdiff_bz,encut,optb88vdw_total_energy,...,ehull,Tc_supercon,exfoliation_energy,density,poisson,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap
0,-0.42762,0.000,NaN,0.0,NaN,60.0,NaN,NaN,650.0,-3.37474,...,0.0423,NaN,NaN,5.956,NaN,8,NaN,NaN,NaN,NaN
1,-0.41596,0.000,NaN,0.0,NaN,65.0,NaN,NaN,1000.0,-5.79186,...,0.0456,NaN,NaN,5.522,NaN,7,NaN,NaN,NaN,NaN
2,0.04847,0.000,NaN,0.0,NaN,60.0,NaN,NaN,1100.0,-4.65924,...,0.3183,NaN,NaN,10.960,NaN,4,NaN,NaN,NaN,NaN
3,-0.44140,0.472,NaN,0.0,NaN,40.0,NaN,NaN,500.0,-0.42496,...,0.0000,NaN,NaN,5.145,NaN,32,NaN,NaN,NaN,NaN
4,-0.71026,0.000,NaN,0.0,NaN,40.0,NaN,NaN,500.0,-3.87823,...,0.0156,NaN,83.78,5.718,0.23,3,48.79,33.05,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75988,-0.30652,0.000,NaN,0.0,NaN,35.0,NaN,NaN,500.0,-2.17686,...,0.0000,NaN,NaN,11.194,NaN,5,NaN,NaN,NaN,NaN
75989,-0.34112,0.000,NaN,0.0,NaN,40.0,NaN,NaN,500.0,-0.10696,...,0.0000,NaN,NaN,8.517,NaN,3,NaN,NaN,NaN,NaN
75990,-0.39352,0.000,NaN,0.0,NaN,35.0,NaN,NaN,500.0,-0.54683,...,0.0000,NaN,NaN,7.460,0.67,5,30.67,-11.44,NaN,NaN
75991,-0.54853,0.000,NaN,0.0,NaN,60.0,NaN,NaN,500.0,-5.48181,...,0.0000,NaN,NaN,8.212,NaN,5,NaN,NaN,NaN,NaN


In [52]:
from pymatgen.core import Composition
df['spg_number'] = df['spg_number'].astype(int)
df['formula'] = [Composition(f) for f in df['formula']]
df['num_spec'] = [len(f.elements) for f in df['formula']]
print(df.shape)
df.head()

(75993, 65)


,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,...,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search,num_spec
0,JVASP-90856,129,P4/nmm,"(Ti, Cu, Si, As)",-0.42762,OptB88vdW,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",NaN,0.0,...,NaN,[],8,NaN,NaN,NaN,NaN,mp-1080455,-As-Cu-Si-Ti,4
1,JVASP-86097,221,Pm-3m,"(Dy, B)",-0.41596,OptB88vdW,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",NaN,0.0,...,NaN,"[OPT-LOPTICS,JVASP-86097.zip,https://ndownload...",7,NaN,NaN,NaN,NaN,mp-568319,-B-Dy,2
2,JVASP-64906,119,I-4m2,"(Be, Os, Ru)",0.04847,OptB88vdW,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",NaN,0.0,...,NaN,"[OPT-LOPTICS,JVASP-64906.zip,https://ndownload...",4,NaN,NaN,NaN,NaN,auid-3eaf68dd483bf4f4,-Be-Os-Ru,3
3,JVASP-98225,14,P2_1/c,"(K, Bi)",-0.44140,OptB88vdW,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",NaN,0.0,...,NaN,[],32,NaN,NaN,NaN,NaN,mp-31104,-Bi-K,2
4,JVASP-10,164,P-3m1,"(V, Se)",-0.71026,OptB88vdW,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",NaN,0.0,...,0.23,"[FD-ELAST,JVASP-10.zip,https://ndownloader.fig...",3,48.79,33.05,0.0,NaN,mp-694,-Se-V,2


In [53]:
col_nans = df.select_dtypes('number').isna().mean(axis=0).sort_values()
col_nans

spg_number                              0.000000
nat                                     0.000000
density                                 0.000000
ehull                                   0.000000
optb88vdw_total_energy                  0.000000
num_spec                                0.000000
optb88vdw_bandgap                       0.000000
formation_energy_peratom                0.000000
kpoint_length_unit                      0.004237
encut                                   0.004250
magmom_outcar                           0.022792
magmom_oszicar                          0.061493
epsx                                    0.313516
epsy                                    0.313516
epsz                                    0.313516
bulk_modulus_kv                         0.686497
shear_modulus_gv                        0.686497
poisson                                 0.689485
n-Seebeck                               0.694472
pkappa                                  0.694472
nkappa              

In [54]:
df['magmom'] = np.where(df.eval('magmom_outcar < 0'), df['magmom_oszicar'], df['magmom_outcar'])
print(df['magmom'].isna().mean())
df['magmom'].quantile(np.linspace(0, 1, 9))

0.022923163975629334


0.000     0.000
0.125     0.000
0.250     0.000
0.375     0.000
0.500     0.000
0.625     0.000
0.750     0.095
0.875     3.724
1.000    64.912
Name: magmom, dtype: float64

In [55]:
clean = df.dropna(axis=1, thresh=0.94 * len(df.index))
clean = clean.drop(columns=['magmom_outcar', 'spg_symbol', 'jid', 'func', 'effective_masses_300K', 'typ', 'spg', 'raw_files', 'reference', 'search', 'elastic_tensor', 'kpoint_length_unit', 'icsd', 'xml_data_link', 'modes', 'encut', 'efg'])
clean

,spg_number,formula,formation_energy_peratom,optb88vdw_bandgap,atoms,optb88vdw_total_energy,ehull,dimensionality,crys,density,nat,num_spec,magmom
0,129,"(Ti, Cu, Si, As)",-0.42762,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",-3.37474,0.0423,3D-bulk,tetragonal,5.956,8,4,0.0
1,221,"(Dy, B)",-0.41596,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",-5.79186,0.0456,3D-bulk,cubic,5.522,7,2,0.0
2,119,"(Be, Os, Ru)",0.04847,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",-4.65924,0.3183,intercalated ion,tetragonal,10.960,4,3,0.0
3,14,"(K, Bi)",-0.44140,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",-0.42496,0.0000,intercalated ion,monoclinic,5.145,32,2,0.0
4,164,"(V, Se)",-0.71026,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",-3.87823,0.0156,2D-bulk,trigonal,5.718,3,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75988,139,"(Ac, Rh, Pb)",-0.30652,0.000,"[[0. 0. 0.] Ac, [2.22044605e-16 2.37450940e+00...",-2.17686,0.0000,3D-bulk,tetragonal,11.194,5,3,0.0
75989,216,"(Pr, Tl, Zn)",-0.34112,0.000,"[[1.71052992 1.71052992 1.71052992] Pr, [5.131...",-0.10696,0.0000,3D-bulk,cubic,8.517,3,3,0.0
75990,139,"(Ba, In, Bi)",-0.39352,0.000,"[[0. 0. 0.] Ba, [4.08371684e+00 4.44089210e-16...",-0.54683,0.0000,intercalated ion,tetragonal,7.460,5,3,0.0
75991,139,"(Tm, Si, Tc)",-0.54853,0.000,"[[0. 0. 0.] Tm, [2.90393031 0. 1.21828...",-5.48181,0.0000,3D-bulk,tetragonal,8.212,5,3,0.0


In [56]:
# clean = clean.query('crys == "cubic"').drop(columns=['crys'])
clean = clean.drop(columns=['crys'])
clean.shape

(75993, 12)

In [57]:
pd.value_counts(clean['dimensionality'])

/tmp/ipykernel_3393550/4219960196.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(clean['dimensionality'])


dimensionality
3D-bulk                  36101
intercalated ion         31682
2D-bulk                   4427
0D-bulk                   1817
1D-bulk                   1320
na                         433
intercalated molecule      213
Name: count, dtype: int64

In [60]:
counts = pd.value_counts(clean['nat'].clip(None, 61)).sort_index()
counts.to_frame().T

/tmp/ipykernel_3393550/3986073793.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  counts = pd.value_counts(clean['nat'].clip(None, 61)).sort_index()


nat,1,2,3,4,5,6,7,8,9,10,...,49,50,52,54,55,56,57,58,60,61
count,374,3829,6915,16740,3731,6205,1258,4995,2065,3541,...,2,23,53,20,2,56,2,21,25,123


In [61]:
clean = clean.query('nat <= 60')

In [62]:
from collections import Counter

els = Counter()

for form in clean['formula']:
    for el in form.elements:
        els[el] += 1

els = pd.Series(els)
filtered = els.sort_values()[-72:]
filtered

Os      954
Tm     1026
U      1155
Tb     1210
Dy     1300
      ...  
Ba     5045
F      5421
S      5465
Li     7609
O     20690
Length: 72, dtype: int64

In [63]:
clean = clean[[all(e in filtered.index for e in f.elements) for f in clean['formula']]]
clean

,spg_number,formula,formation_energy_peratom,optb88vdw_bandgap,atoms,optb88vdw_total_energy,ehull,dimensionality,density,nat,num_spec,magmom
0,129,"(Ti, Cu, Si, As)",-0.42762,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",-3.37474,0.0423,3D-bulk,5.956,8,4,0.0
1,221,"(Dy, B)",-0.41596,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",-5.79186,0.0456,3D-bulk,5.522,7,2,0.0
2,119,"(Be, Os, Ru)",0.04847,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",-4.65924,0.3183,intercalated ion,10.960,4,3,0.0
3,14,"(K, Bi)",-0.44140,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",-0.42496,0.0000,intercalated ion,5.145,32,2,0.0
4,164,"(V, Se)",-0.71026,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",-3.87823,0.0156,2D-bulk,5.718,3,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75980,164,"(Nd, Mg, Sb)",-0.69038,0.000,"[[0. 0. 0.] Nd, [2.3394364 1.35070495 2.69383...",-1.53096,0.0000,3D-bulk,5.116,5,3,0.0
75983,216,"(Yb, Pr, Pd)",-0.56635,0.000,"[[5.25036997 5.25036997 5.25036997] Yb, [1.750...",-1.70796,0.0000,3D-bulk,8.139,3,3,0.0
75984,216,"(Tb, Tl, Zn)",-0.29921,0.000,"[[1.66349368 1.66349368 1.66349368] Tb, [4.990...",0.05135,0.0000,3D-bulk,9.666,3,3,0.0
75989,216,"(Pr, Tl, Zn)",-0.34112,0.000,"[[1.71052992 1.71052992 1.71052992] Pr, [5.131...",-0.10696,0.0000,3D-bulk,8.517,3,3,0.0


In [64]:
pd.value_counts(clean['num_spec']).sort_index()

/tmp/ipykernel_3393550/2605059850.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(clean['num_spec']).sort_index()


num_spec
1      776
2    17612
3    40095
4    10388
5     1494
6       97
7        5
Name: count, dtype: int64

In [65]:
clean = clean.query('num_spec <= 6')
clean

,spg_number,formula,formation_energy_peratom,optb88vdw_bandgap,atoms,optb88vdw_total_energy,ehull,dimensionality,density,nat,num_spec,magmom
0,129,"(Ti, Cu, Si, As)",-0.42762,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",-3.37474,0.0423,3D-bulk,5.956,8,4,0.0
1,221,"(Dy, B)",-0.41596,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",-5.79186,0.0456,3D-bulk,5.522,7,2,0.0
2,119,"(Be, Os, Ru)",0.04847,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",-4.65924,0.3183,intercalated ion,10.960,4,3,0.0
3,14,"(K, Bi)",-0.44140,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",-0.42496,0.0000,intercalated ion,5.145,32,2,0.0
4,164,"(V, Se)",-0.71026,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",-3.87823,0.0156,2D-bulk,5.718,3,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75980,164,"(Nd, Mg, Sb)",-0.69038,0.000,"[[0. 0. 0.] Nd, [2.3394364 1.35070495 2.69383...",-1.53096,0.0000,3D-bulk,5.116,5,3,0.0
75983,216,"(Yb, Pr, Pd)",-0.56635,0.000,"[[5.25036997 5.25036997 5.25036997] Yb, [1.750...",-1.70796,0.0000,3D-bulk,8.139,3,3,0.0
75984,216,"(Tb, Tl, Zn)",-0.29921,0.000,"[[1.66349368 1.66349368 1.66349368] Tb, [4.990...",0.05135,0.0000,3D-bulk,9.666,3,3,0.0
75989,216,"(Pr, Tl, Zn)",-0.34112,0.000,"[[1.71052992 1.71052992 1.71052992] Pr, [5.131...",-0.10696,0.0000,3D-bulk,8.517,3,3,0.0


In [66]:
clean['magmom'] = clean['magmom'].fillna(0)
clean = clean.rename(columns={'optb88vdw_bandgap': 'bandgap', 'nat': 'num_atoms', 'spg_number': 'space_group', 'formation_energy_peratom': 'e_form', 'optb88vdw_total_energy': 'e_total'})
clean

/tmp/ipykernel_3393550/3805300746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['magmom'] = clean['magmom'].fillna(0)


,space_group,formula,e_form,bandgap,atoms,e_total,ehull,dimensionality,density,num_atoms,num_spec,magmom
0,129,"(Ti, Cu, Si, As)",-0.42762,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",-3.37474,0.0423,3D-bulk,5.956,8,4,0.0
1,221,"(Dy, B)",-0.41596,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",-5.79186,0.0456,3D-bulk,5.522,7,2,0.0
2,119,"(Be, Os, Ru)",0.04847,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",-4.65924,0.3183,intercalated ion,10.960,4,3,0.0
3,14,"(K, Bi)",-0.44140,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",-0.42496,0.0000,intercalated ion,5.145,32,2,0.0
4,164,"(V, Se)",-0.71026,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",-3.87823,0.0156,2D-bulk,5.718,3,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75980,164,"(Nd, Mg, Sb)",-0.69038,0.000,"[[0. 0. 0.] Nd, [2.3394364 1.35070495 2.69383...",-1.53096,0.0000,3D-bulk,5.116,5,3,0.0
75983,216,"(Yb, Pr, Pd)",-0.56635,0.000,"[[5.25036997 5.25036997 5.25036997] Yb, [1.750...",-1.70796,0.0000,3D-bulk,8.139,3,3,0.0
75984,216,"(Tb, Tl, Zn)",-0.29921,0.000,"[[1.66349368 1.66349368 1.66349368] Tb, [4.990...",0.05135,0.0000,3D-bulk,9.666,3,3,0.0
75989,216,"(Pr, Tl, Zn)",-0.34112,0.000,"[[1.71052992 1.71052992 1.71052992] Pr, [5.131...",-0.10696,0.0000,3D-bulk,8.517,3,3,0.0


In [67]:
clean.sort_values('e_form')

,space_group,formula,e_form,bandgap,atoms,e_total,ehull,dimensionality,density,num_atoms,num_spec,magmom
2481,165,"(La, F)",-4.42304,6.245,[[2.46331072e+00 1.30683253e-11 1.83865753e+00...,-4.90293,0.0000,3D-bulk,5.914,24,2,0.000
35556,136,"(Yb, F)",-4.38553,6.992,"[[0. 0. 0.] Yb, [2.53513021 2.53513021 1.74189...",-3.90615,0.0000,3D-bulk,7.826,6,2,0.000
31630,194,"(La, F)",-4.37937,7.036,"[[2.08572307 1.20419479 1.84094223] La, [ 2.08...",-4.85926,0.0437,3D-bulk,5.863,8,2,0.000
6750,62,"(Er, F)",-4.37655,8.094,"[[4.12580907 3.96475409 5.11317578] Er, [1.931...",-4.68192,0.0000,3D-bulk,7.919,16,2,0.000
26613,62,"(Ho, F)",-4.37421,8.087,"[[2.46166195 0.83066565 5.14043582] Ho, [4.097...",-4.69240,0.0000,3D-bulk,7.728,16,2,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
759,194,(N),5.28308,2.340,"[[1.82006459 1.05081681 4.44819094] N, [ 1.820...",-1.57863,0.0000,intercalated ion,0.683,2,1,3.900
42239,194,(N),5.28308,2.340,[[-1.82006437e-06 2.10163023e+00 1.48273011e...,-1.57863,0.0000,intercalated ion,0.683,2,1,3.900
20291,225,(N),5.28329,2.557,[[2.57144673e+00 4.33717332e-04 2.57057929e+00...,-1.57841,0.0000,intercalated ion,0.684,1,1,7.800
36359,225,(N),5.28329,2.557,[[2.57144672e+00 4.33712907e-04 2.57057930e+00...,-1.57841,0.0000,intercalated ion,0.684,1,1,7.800


I'm mostly removing the last N columns so the dataset size splits evenly. The high-formation-energy
values are generally less useful.

In [69]:
from sympy import factorint
n_data = len(clean.index)
for power in range(1, 11):
    bs = 2 ** power
    rem = n_data % bs
    factorstr = '\t'.join((f'{k}^{v}' for k, v in factorint(n_data - rem).items()))
    print(f'{bs:>4}\t{rem:>4}\t{factorstr}')

   2	   0	2^1	7^2	719^1
   4	   2	2^2	5^1	13^1	271^1
   8	   6	2^3	8807^1
  16	  14	2^4	7^1	17^1	37^1
  32	  30	2^5	31^1	71^1
  64	  62	2^8	5^2	11^1
 128	  62	2^8	5^2	11^1
 256	  62	2^8	5^2	11^1
 512	 318	2^9	137^1
1024	 830	2^12	17^1


In [71]:
max_e_form = clean.sort_values('e_form').iloc[n_data - 62 - 1]['e_form']
print(max_e_form)
clean = clean.query('e_form <= @max_e_form')
clean

3.30373


,space_group,formula,e_form,bandgap,atoms,e_total,ehull,dimensionality,density,num_atoms,num_spec,magmom
0,129,"(Ti, Cu, Si, As)",-0.42762,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",-3.37474,0.0423,3D-bulk,5.956,8,4,0.0
1,221,"(Dy, B)",-0.41596,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",-5.79186,0.0456,3D-bulk,5.522,7,2,0.0
2,119,"(Be, Os, Ru)",0.04847,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",-4.65924,0.3183,intercalated ion,10.960,4,3,0.0
3,14,"(K, Bi)",-0.44140,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",-0.42496,0.0000,intercalated ion,5.145,32,2,0.0
4,164,"(V, Se)",-0.71026,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",-3.87823,0.0156,2D-bulk,5.718,3,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75980,164,"(Nd, Mg, Sb)",-0.69038,0.000,"[[0. 0. 0.] Nd, [2.3394364 1.35070495 2.69383...",-1.53096,0.0000,3D-bulk,5.116,5,3,0.0
75983,216,"(Yb, Pr, Pd)",-0.56635,0.000,"[[5.25036997 5.25036997 5.25036997] Yb, [1.750...",-1.70796,0.0000,3D-bulk,8.139,3,3,0.0
75984,216,"(Tb, Tl, Zn)",-0.29921,0.000,"[[1.66349368 1.66349368 1.66349368] Tb, [4.990...",0.05135,0.0000,3D-bulk,9.666,3,3,0.0
75989,216,"(Pr, Tl, Zn)",-0.34112,0.000,"[[1.71052992 1.71052992 1.71052992] Pr, [5.131...",-0.10696,0.0000,3D-bulk,8.517,3,3,0.0


In [72]:
clean.to_pickle('precomputed/jarvis_dft3d_cleaned/dataframe.pkl')

In [ ]:
clean = pd.read_pickle('precomputed/jarvis_dft3d_cleaned/dataframe.pkl')

In [ ]:
pd.value_counts(clean['space_group'])

In [ ]:
elements = set()
for struct in clean['atoms']:
    elements.update(set(struct.elements))

print(len(elements))
# print(elements)

sorted_elements = sorted(elements, key=lambda e: e.number)
elem_vals = [elem.symbol for elem in sorted_elements]
print(' '.join(elem_vals))